In [1]:
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import torch

In [2]:
!nvidia-smi

Thu Apr  4 15:02:08 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:00:05.0 Off |                  Off |
| 30%   42C    P8    24W / 300W |      1MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
notes = pd.read_csv('data/notes_with_interval.csv')

notes.head()

,note_id,id,note_type,note_seq,charttime,text,icu_death,interval
0,17915608-RR-64,20008098,RR,64,1975-02-06 16:23:00,EXAMINATION: CHEST (PORTABLE AP)\n\nINDICATIO...,0,3
1,17915608-RR-65,20008098,RR,65,1975-02-07 10:50:00,EXAMINATION: CHEST (PORTABLE AP)\n\nINDICATIO...,0,2
2,17915608-RR-66,20008098,RR,66,1975-02-07 20:17:00,INDICATION: ___ year old man s/p RUL lobectom...,0,2
3,17915608-RR-67,20008098,RR,67,1975-02-08 12:20:00,INDICATION: ___ year old man s/p VATS to Open...,0,1
4,17915608-RR-68,20008098,RR,68,1975-02-09 07:26:00,INDICATION: ___ year old man s/p open RUL lob...,0,1


In [4]:
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)

print(f'device: {device}')

device: cuda


In [6]:
test_ids = [20195689, 22962803]

notes = notes[notes['id'].isin(test_ids)]

notes.head()

,note_id,id,note_type,note_seq,charttime,text,icu_death,interval
453,15842769-RR-15,20195689,RR,15,2038-07-05 00:42:00,HISTORY: ___ male with pneumocephalus; thin-s...,1,14
454,15842769-RR-16,20195689,RR,16,2038-07-06 11:12:00,INDICATION: ___ with unexplained persistent p...,1,12
455,15842769-RR-17,20195689,RR,17,2038-07-08 14:41:00,INDICATION: ___ male with pneumocephalus Eval...,1,10
456,15842769-RR-18,20195689,RR,18,2038-07-09 13:37:00,INDICATION: ___ male with pneumocephalus.\n\n...,1,9
457,15842769-RR-20,20195689,RR,20,2038-07-09 12:03:00,"INDICATION: ___ man with pneumocephalus, eval...",1,9


In [7]:
notes.id.value_counts()

20195689    18
22962803     1
Name: id, dtype: int64

In [ ]:
for id in test_ids:
    

In [8]:
tokenized_documents = tokenizer(
    notes.text.to_list(),
    padding=True, truncation=True, 
    max_length=512, return_tensors="pt"
    )

tokenized_documents = tokenized_documents.to(device)

In [9]:
tokenized_documents.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [10]:
outputs = model(**tokenized_documents)

In [19]:
outputs.pooler_output.shape

torch.Size([19, 768])

In [20]:
len(outputs)

2

In [22]:
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [25]:
len(tokenized_documents.input_ids)

19